<a href="https://colab.research.google.com/github/michaelgadda/OLSViaCoordinateDescent/blob/main/OLSViaCoordinateDescent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn import datasets
from sklearn.linear_model import Lasso
from sklearn.linear_model import lasso_path
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np
from copy import deepcopy

In [30]:
class OLSViaCoordDesc:
  def __init__(self, epochs=10000, epsilon=1, eta=.1):
    self.epochs = epochs
    self.epsilon = epsilon
    self.eta = eta
    self.theta = None
    self.OG_THETA = None
    self.intercept = None

  def preappend_bias_term(self, X):
    inter = np.ones(X.shape[0])
    X_new = np.column_stack((inter, X))
    return X_new



  def fit(self, X, Y):
    X = self.preappend_bias_term(X)
    X = X / np.sqrt(np.sum(np.square(X), axis=0))
    print(X[0,0])


    print(X.shape)

    Y = Y.reshape(-1,1)
    n_cols = X.shape[1]
    print(n_cols)
    n_rows = X.shape[0]
    self.theta = np.zeros(n_cols).reshape(-1,1)

    best_mse = None
    print(self.theta)
    dw = np.zeros(n_cols)
    best_theta = np.zeros(n_cols)
    for epoch in range(self.epochs):
      print(epoch)
      y_pred = X @ self.theta
      MSE = (np.sum((y_pred - Y)**2))/n_rows
      print(f'MSE: {MSE} Best MSE: {best_mse}')
      if best_mse == None:
        best_mse = MSE
      elif best_mse <= MSE:
        self.theta = best_theta
        break
      elif best_mse > MSE:
        best_theta = deepcopy(self.theta)
        best_mse = MSE

      for column in range(n_cols):
        X_col = X[:,column].reshape(-1,1)
        theta_j = self.theta[column]
        if column == 0:
          temp_theta = self.theta[column+1:].reshape(-1,1) #arr[:idx] + arr[idx + 1:]
          temp_X = X[:,column+1:]
        elif column + 1 == n_cols:
          temp_theta = self.theta[:column].reshape(-1,1) #arr[:idx] + arr[idx + 1:]
          temp_X = X[:, :column]
        else:
          temp_theta = np.vstack((self.theta[:column], self.theta[column+1:])).reshape(-1,1) #arr[:idx] + arr[idx + 1:]
          temp_X = np.hstack((X[:, :column], X[:,column+1:]))

        y_pred = temp_X @ temp_theta
        rj = (Y - y_pred)

        dw_j = X_col.T @ rj
        self.theta[column] = dw_j
        print(f'Y: {np.sum(y)}, X: {np.sum(X, axis=0)}')
    self.theta[0] = np.sum(Y)/n_rows - self.theta[1:].T @ np.sum(X[:,1:], axis = 0)/n_rows


  def test_against_ols(self, X,y):
    inter = np.ones(X.shape[0])
    X_new = np.column_stack((inter, X))
    X_Normalized = X_new / np.sqrt(np.sum(np.square(X_new), axis=0))

    w = np.zeros(X_Normalized.shape[1]).reshape(-1,1)

    for iteration in range(100):
        r = y - X_Normalized.dot(w)


        for j in range(len(w)):
            print(f"This is r {r}")
            r = r + (X_Normalized[:, j].reshape(-1,1) @ w[j].reshape(-1,1))

            w[j] = X_Normalized[:, j].dot(r)
            r = r - (X_Normalized[:, j].reshape(-1,1) @ w[j].reshape(-1,1))
    return w



  def predict(self, X):
      X = self.preappend_bias_term(X)
      print(self.theta.shape, X.shape)
      return X@self.theta

In [31]:
diabetes_dataset = datasets.load_diabetes()

X = diabetes_dataset.data[:, :8]
y = diabetes_dataset.target.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 99)

In [32]:
reg_X = X_train / np.sqrt(np.sum(np.square(X_train), axis=0))
LR = LinearRegression(fit_intercept=True)
myLR = OLSViaCoordDesc()
inter = np.ones(X_train.shape[0])
X_new = np.column_stack((inter, X_train))
inter_test = np.ones(X_test.shape[0])
X_test_new = np.column_stack((inter_test, X_test))
LR.fit(reg_X, y_train)
test_against_weights = myLR.test_against_ols(X_train, y_train)
myLR.fit(X_train, y_train)

print(f'\n\n SK: {LR.coef_}, \n\n MY: {myLR.intercept, myLR.theta}, \n\n HIS: {test_against_weights}')

Streaming output truncated to the last 5000 lines.
 [ 5.66385394e+01]
 [ 3.11217811e+01]
 [ 1.64752161e+01]
 [ 8.15207836e+01]
 [-4.94107240e+00]
 [-1.03397238e+02]
 [ 1.16824566e+02]
 [-1.49173414e+02]
 [-1.60948340e+01]
 [-1.52776424e+01]
 [ 4.00151642e-02]
 [ 6.75301085e+01]
 [ 7.69759387e+01]
 [ 6.04732824e+01]
 [ 4.17071787e+01]
 [ 6.06093568e+00]
 [ 6.94728259e+01]
 [ 2.81052998e+01]
 [ 8.06752358e+01]
 [ 1.26095997e+01]
 [-6.30710399e+01]
 [ 2.60349317e+01]
 [-8.01100041e+00]
 [ 4.07486477e+01]
 [ 6.58952805e+01]
 [-9.63302622e+01]
 [ 6.69559344e+00]
 [-4.42549399e+01]
 [-2.39399875e+01]
 [-3.79623564e+01]
 [-1.83001070e+01]
 [ 1.61602684e+02]
 [-1.14747638e+02]
 [ 1.60475095e+01]
 [-3.04197110e+01]
 [ 1.71272550e-02]
 [-8.93560344e+00]
 [ 3.28737126e+01]
 [ 6.86481982e+01]
 [ 3.75795117e+00]
 [ 5.40563119e+01]
 [ 3.31086960e+01]
 [-2.50751326e+01]
 [ 8.49358260e+01]
 [-5.34530701e+01]
 [-7.84222250e+01]
 [-2.81793092e+01]
 [ 1.21530650e+01]
 [-1.72882749e+01]
 [ 2.20587559e+01]

In [34]:
sk_mse = mean_squared_error(LR.predict(X_test), y_test)
my_mse = mean_squared_error(myLR.predict(X_test), y_test)
sk_pred_vals = LR.predict(X_test)
my_pred_vals = myLR.predict(X_test)
sk_r2 = r2_score(sk_pred_vals, y_test)
my_r2 = r2_score(my_pred_vals, y_test)
print(f'Sklearn MSE: {sk_mse} My MSE: {my_mse} \n Sklearn R2: {sk_r2} My R2: {my_r2}')


(9, 1) (111, 9)
(9, 1) (111, 9)
Sklearn MSE: 3296.237468630039 My MSE: 3296.237469444605 
 Sklearn R2: -0.49320574426144037 My R2: -0.49320572010176744
